In [381]:
import numpy as np
import cv2 as cv
from skimage import filters, morphology, measure, segmentation
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import savemat
import pandas as pd
import scipy as sp
import os

In [382]:
CYCLIC_COLUMNS = ["PSVTime", "PSV", "PDVTime", "PDV", "EDVTime", "EDV", "ISVTime", "ISV", "MBF1", "MBF2", "SysTime", "DiasTime", "OSI", "WindowTime"]
TIME_SERIES_COLUMNS = ["Time","BloodFlow","Filtered Blood Flow", "Shear", "Filtered Shear", "Velocity", "Diameter"]
ANALYSIS_SETTINGS_COLUMNS = ["Peak Height Threshold", "Cycle Duration", "Peak Count", "EpochEndTime", "Velocity Calibration", "Time Calibration", "Distance Calibration", "Zero Velocity Row Position"]

In [383]:
def subj_file(subj_str: str, trial_type: str, data_type: str, return_video_file = False):
    folder = f"bloodflow_data/{subj_str}"
    filename = None
    files = None
    
    if trial_type in ['baseline', 'base', 'basline']:
        files = [file_name for file_name in os.listdir(folder) if 'baseline' in file_name or 'basline' in file_name or 'base' in file_name]

    elif trial_type in ['post-oc', 'post-occ', 'post-occlusion', 'post']:
        files = [file_name for file_name in os.listdir(folder) if 'post' in file_name]

    else:
        raise ValueError(f"{trial_type} is not an acceptable parameter for trial_type")
    
    if data_type in ['diameter', 'diam']:
        files = [file_name for file_name in files if 'diam' in file_name]
        filename = files[0][:-4]
    elif data_type in ['time', 'time_series', 'time series', 'cyclic'] :
        files = [file_name for file_name in files if 'flow' in file_name or 'doppler' in file_name]
        filename = files[0][:str.index(files[0], '_')]
        
    else:
        raise ValueError(f"{data_type} is not an acceptable parameter for data_type")
    
    if return_video_file:
        folder = "raw_video"
    return f"{folder}/{filename}", data_type

In [384]:
def get_px_to_cm(frame):
    print("Select ROI that surrounds the reference scale")
    r = cv.selectROI(frame)
    cv.destroyAllWindows()
    px_to_cm = float(input("How many centimeters is this? "))
    px_to_cm /= r[3]
    return px_to_cm

In [385]:
subj_no = 14

In [386]:
subj_name = f"ASCI1{subj_no:02}"

baseline_video = subj_file(subj_name, 'baseline', 'diam', return_video_file=True)
baseline = subj_file(subj_name, 'baseline', 'diam')

post_video = subj_file(subj_name, 'post', 'diam', return_video_file=True)
post = subj_file(subj_name, 'post', 'diam')
subj = [baseline, post]
subj

[('bloodflow_data/ASCI114/ASCI114 - baseline long diam 20210312141740339',
  'diam'),
 ('bloodflow_data/ASCI114/ASCI114 - post-occ long diam 20210312142448015',
  'diam')]

In [387]:
def get_frame(video):
    capture = cv.VideoCapture(f"{video[0]}.avi")

    ret, frame = capture.read()

    if not ret:
        exit(f"File: \"{video[0]}.avi\" not found.")
        
    return frame

In [388]:
def save(subj, px_to_cm):
    print(subj[0])
    data = np.load(f"{subj[0]}.npy")
    diameter = data['diameters']
    roi = data['roi']
    data.close()
        
    with open(f"{subj[0]}.npy", 'wb') as f:
        np.savez(f, diameters = diameter, roi = roi, px_to_cm = px_to_cm)


In [389]:
def preview(subj):
    data = np.load(f"{subj[0]}.npy")
    diameter = data['diameters']
    roi = data['roi']
    px_to_cm = data['px_to_cm']
    data.close()
    
    print(f"{np.nanmean(diameter) * px_to_cm:0.3f} cm")

In [390]:
repeats = 3

frames = [get_frame(baseline_video), get_frame(post_video)]

for n in range(2):
    px_to_cm = np.empty(repeats)

    for i in range(repeats):
        px_to_cm[i] = get_px_to_cm(frames[n])
        print(f"Pixel to cm: {px_to_cm[i]}\n")

    save(subj[n], px_to_cm.mean())
    preview(subj[n])

Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

bloodflow_data/ASCI114/ASCI114 - baseline long diam 20210312141740339
0.297 cm
Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

Select ROI that surrounds the reference scale
Pixel to cm: 0.008305647840531562

bloodflow_data/ASCI114/ASCI114 - post-occ long diam 20210312142448015
0.310 cm
